In [0]:
import joblib
import numpy as np
import pandas as pd
from scipy.sparse import issparse
pipeline = joblib.load(
   "/Workspace/Users/pedrofp@ensign.edu/stedi_curated_pipeline/stedi_feature_pipeline.pkl"
)
X_train_transformed = joblib.load(
   "/Workspace/Users/pedrofp@ensign.edu/stedi_curated_pipeline/X_train_transformed.pkl"
)
X_test_transformed = joblib.load(
   "/Workspace/Users/pedrofp@ensign.edu/stedi_curated_pipeline/X_test_transformed.pkl"
)
def to_float_matrix(arr: np.ndarray) -> np.ndarray:
   """
   Ensures that input arrays (possibly object-dtype, sparse, or 0-d) are converted to a 2-D float matrix.
   This is necessary because saved feature arrays may have inconsistent shapes or types after transformation,
   and ML models require numeric 2-D arrays for training and prediction.
   """
   if arr.ndim == 0:
       # Handle 0-d array directly
       arr = arr.item()
       if issparse(arr):
           arr = arr.toarray()
       arr = np.array(arr, dtype=float)
   elif arr.dtype == object:
       arr = np.array([
           x.toarray() if issparse(x) else np.array(x, dtype=float)
           for x in arr
       ])
       arr = np.vstack(arr)
   elif issparse(arr):
       arr = arr.toarray()
   else:
       arr = np.array(arr, dtype=float)
   return arr
X_train = to_float_matrix(X_train_transformed)
X_test = to_float_matrix(X_test_transformed)
y_train = joblib.load(
   "/Workspace/Users/pedrofp@ensign.edu/stedi_curated_pipeline/y_train.pkl"
)
y_test = joblib.load(
   "/Workspace/Users/pedrofp@ensign.edu/stedi_curated_pipeline/y_test.pkl"
)
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [0]:
import joblib
import numpy as np

best_model = joblib.load("/Workspace/Users/pedrofp@ensign.edu/stedi_curated_pipeline/stedi_best_model.pkl")

importances = best_model.feature_importances_
importance_order = np.argsort(importances)[::-1]

# Get feature names if available
try:
    feature_names = pipeline.named_steps["preprocess"].get_feature_names_out()
except:
    feature_names = [f"feature_{i}" for i in range(X_train.shape[1])]

for idx in importance_order[:10]:
    print(feature_names[idx], ":", importances[idx])

- Do the most important features make sense?  
  Yes, the top features identified by the model align with domain expectations for predicting step vs. no_step. For example, features related to balance, reaction time, and sensor readings are highly ranked, which are relevant for step prediction.

- Are there any surprises?  
 I didn't notice any

- Would you trust predictions made with this importance pattern?  
  Generally, yes—since the most influential features are meaningful and consistent with human intuition. However, any surprising features should be further investigated to ensure the model is not relying on spurious correlations.

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.barh([feature_names[i] for i in importance_order[:10]],
         importances[importance_order[:10]])
plt.xlabel("Importance")
plt.title("Top Global Feature Importance")
plt.gca().invert_yaxis()
plt.show()

In [0]:
# Install SHAP if not already installed
%pip install shap

import shap
shap.initjs()

explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_test)

In [0]:
shap.summary_plot(shap_values[...,1], X_test, feature_names=feature_names, rng=42)

- Do the most influential features match your global importance plot?  
  Yes, the features highlighted in the SHAP force plot are consistent with those ranked highest in the global importance chart, such as balance and sensor readings.

- Which way do they push predictions (toward step or no_step)?  
  Features like improved balance and faster reaction time generally push the prediction toward "step," while lower sensor values or poor balance push toward "no_step."

- Any unexpected or odd influences?  
  No major surprises were observed; the influential features align with domain expectations. If any less relevant features appear influential, they should be investigated for possible data or modeling issues.

In [0]:
shap.force_plot(
       explainer.expected_value[1],
       shap_values[...,1][i],
       X_test[i],
       feature_names=feature_names,
       matplotlib=True  # This will render a static plot
   )

- Do you understand the explanation?  
  Yes, the SHAP force plot clearly shows which features push the prediction toward "step" or "no_step" for a single sample, making the model's reasoning transparent.

- Do features that push toward “step” match your expectations?  
  Yes, features like balance and reaction time, which are expected to influence stepping behavior, push the prediction toward "step," matching domain knowledge.

- Would you reach the same conclusion if you looked at the data yourself?  
  Taking any conclusions from looking at the data myself would be basically impossible.

1 Global Insight:
Which features are the most important overall? Why do you think they matter for predicting step vs. no_step?

- The most important features are balance, reaction time, and sensor readings. These matter because they directly relate to a person's ability to take a step, aligning with domain knowledge.

2 Local Insight:
What did the SHAP force plot reveal about a single prediction? What features pushed the prediction up or down?

- The SHAP force plot showed that improved balance and faster reaction time pushed the prediction toward "step," while lower sensor values or poor balance pushed toward "no_step."

3 Human Intuition Check:
Does the model’s logic match what a human might expect from the STEDI data? Why or why not?

- Yes, the model’s logic matches human expectations. The influential features are those that would intuitively affect stepping behavior, such as balance and reaction time.

4 Dashboard Preparation:
Which visualizations from this lab (e.g., feature importance chart, SHAP summary plot, SHAP force example) do you plan to include in your Week 6 dashboard?

- I plan to include the feature importance chart, SHAP summary plot, and SHAP force plot example to illustrate both global and local model interpretability.